<a href="https://colab.research.google.com/github/roshansrini26/object_lane_detection/blob/main/lane_object_detection_for_straight_curvy_roads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade opencv-python opencv-python-headless

In [ ]:
import cv2
import numpy as np
import math
import copy

In [ ]:
def preprocessing(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gblur = cv2.GaussianBlur(gray,(5, 5),0)
    thresh = cv2.threshold(gblur,150,255,cv2.THRESH_BINARY)[1]
    return thresh

In [ ]:
def regionOfInterest(img, polygon):
    mask = np.zeros_like(img)
    x1, y1 = polygon[0]
    x2, y2 = polygon[1]
    x3, y3 = polygon[2]
    x4, y4 = polygon[3]
    m1 = (y2-y1)/(x2-x1)
    m2 = (y3-y2)/(x3-x2)
    m3 = (y4-y3)/(x4-x3)
    m4 = (y4-y1)/(x4-x1)
    b1 = y1 - m1*x1
    b2 = y2 - m2*x2
    b3 = y3 - m3*x3
    b4 = y4 - m4*x4

    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if i>=m1*j+b1 and i>=m2*j+b2 and i>=m3*j+b3 and i<=m4*j+b4:
                mask[i][j] = 1

    masked_img = np.multiply(mask, img)
    return masked_img

In [ ]:
def slopeIntercept(line):
    m = (line[1][1]-line[0][1])/(line[1][0]-line[0][0])
    b = line[1][1] - m*line[1][0]
    return m, b

In [ ]:
def removeCloseLines(linelist, m):
    linelist_copy = copy.deepcopy(linelist)

    for line in linelist:
        m1, _ = slopeIntercept(line)
        if abs(m-m1)<=0.5:
            linelist_copy.remove(line)

    return linelist_copy

In [ ]:
def lineDetection(img, masked_img, solid_line_previous, dashed_line_previous):
    img_copy = copy.deepcopy(img)
    height, width = masked_img.shape
    linesP = cv2.HoughLinesP(masked_img, 1, np.pi/180, 50, None, 30, 20)
    linelist = linesP.tolist()
    linelist = [tuple((line[0][:2], line[0][2:])) for line in linelist]
    line_length = []
    for line in linelist:
        line_length.append(math.dist(line[0], line[1]))

    try:
        solid_line = linelist[line_length.index(max(line_length))]
        linelist.remove(solid_line)
    except ValueError:
        solid_line = solid_line_previous

    m, b = slopeIntercept(solid_line)
    linelist = removeCloseLines(linelist, m)
    initial = (int((height*0.6-b)/m), int(height*0.6))
    final = (int((height-b)/m), height)
    detected_line = cv2.line(img_copy, initial, final, (0,255,0), 5)

    line_length = []

    for line in linelist:
        line_length.append(math.dist(line[0], line[1]))

    try:
        dashed_line = linelist[line_length.index(max(line_length))]
    except ValueError:
        dashed_line = dashed_line_previous

    m, b = slopeIntercept(dashed_line)
    initial = (int((height*0.6-b)/m), int(height*0.6))
    final = (int((height-b)/m), height)
    detected_line = cv2.line(detected_line, initial, final, (0,0,255), 5)

    return detected_line, solid_line, dashed_line

In [ ]:
video = cv2.VideoCapture("straight_lane.mp4")

In [ ]:
video = cv2.VideoCapture("straight_lane.mp4")
out = cv2.VideoWriter('straight_lane_detection.avi', cv2.VideoWriter_fourcc(*'MJPG'), 25, (960,540))
solid_line_previous = None
dashed_line_previous = None
print("Generating video output...\n")

while True:
    isTrue, img = video.read()
    if isTrue == False:
        break
    processed_img = preprocessing(img)
    height, width = processed_img.shape
    polygon = [(int(width*0.1), height), (int(width*0.45), int(height*0.6)), (int(width*0.55), int(height*0.6)), (int(0.95*width), height)]
    masked_img = regionOfInterest(processed_img, polygon)
    detected_lines, solid_line, dashed_line = lineDetection(img, masked_img, solid_line_previous, dashed_line_previous)
    solid_line_previous = solid_line
    dashed_line_previous = dashed_line
    out.write(detected_lines)

# Release video objects
out.release()
video.release()
cv2.destroyAllWindows()  # Close all OpenCV windows
print("Video output generated.\n")


Generating video output...

Video output generated.



In [ ]:
# Download the MobileNet-SSD model files
!wget -O deploy.prototxt https://raw.githubusercontent.com/chuanqi305/MobileNet-SSD/master/deploy.prototxt
!wget -O mobilenet_iter_73000.caffemodel https://github.com/chuanqi305/MobileNet-SSD/raw/master/mobilenet_iter_73000.caffemodel

--2024-10-06 12:35:28--  https://raw.githubusercontent.com/chuanqi305/MobileNet-SSD/master/deploy.prototxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44667 (44K) [text/plain]
Saving to: ‘deploy.prototxt’

deploy.prototxt     100%[===================>]  43.62K  --.-KB/s    in 0.007s  

2024-10-06 12:35:28 (6.39 MB/s) - ‘deploy.prototxt’ saved [44667/44667]

--2024-10-06 12:35:28--  https://github.com/chuanqi305/MobileNet-SSD/raw/master/mobilenet_iter_73000.caffemodel
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chuanqi305/MobileNet-SSD/master/mobilenet_iter_73000.caffemodel [following]
--

In [ ]:
# Load the pre-trained MobileNet-SSD model
net = cv2.dnn.readNetFromCaffe('deploy.prototxt', 'mobilenet_iter_73000.caffemodel')

In [ ]:
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

In [ ]:
def preprocessing(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gblur = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.threshold(gblur, 150, 255, cv2.THRESH_BINARY)[1]
    return thresh

In [ ]:
def regionOfInterest(img, polygon):
    mask = np.zeros_like(img)
    x1, y1 = polygon[0]
    x2, y2 = polygon[1]
    x3, y3 = polygon[2]
    x4, y4 = polygon[3]
    m1 = (y2 - y1) / (x2 - x1)
    m2 = (y3 - y2) / (x3 - x2)
    m3 = (y4 - y3) / (x4 - x3)
    m4 = (y4 - y1) / (x4 - x1)
    b1 = y1 - m1 * x1
    b2 = y2 - m2 * x2
    b3 = y3 - m3 * x3
    b4 = y4 - m4 * x4

    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if i >= m1 * j + b1 and i >= m2 * j + b2 and i >= m3 * j + b3 and i <= m4 * j + b4:
                mask[i][j] = 1

    masked_img = np.multiply(mask, img)
    return masked_img

In [ ]:
# Function to find slope and y-intercept of a line
def slopeIntercept(line):
    m = (line[1][1] - line[0][1]) / (line[1][0] - line[0][0])
    b = line[1][1] - m * line[1][0]
    return m, b

In [ ]:
# Function to remove multiple lines detected on the same lane side
def removeCloseLines(linelist, m):
    linelist_copy = copy.deepcopy(linelist)

    for line in linelist:
        m1, _ = slopeIntercept(line)
        if abs(m - m1) <= 0.5:
            linelist_copy.remove(line)

    return linelist_copy

In [ ]:
# Function that draws lines on the image
def lineDetection(img, masked_img, solid_line_previous, dashed_line_previous):
    img_copy = copy.deepcopy(img)
    height, width = masked_img.shape
    linesP = cv2.HoughLinesP(masked_img, 1, np.pi / 180, 50, None, 30, 20)
    linelist = linesP.tolist()
    linelist = [tuple((line[0][:2], line[0][2:])) for line in linelist]
    line_length = []
    for line in linelist:
        line_length.append(math.dist(line[0], line[1]))

    try:
        solid_line = linelist[line_length.index(max(line_length))]
        linelist.remove(solid_line)
    except ValueError:
        solid_line = solid_line_previous

    m, b = slopeIntercept(solid_line)
    linelist = removeCloseLines(linelist, m)
    initial = (int((height * 0.6 - b) / m), int(height * 0.6))
    final = (int((height - b) / m), height)
    detected_line = cv2.line(img_copy, initial, final, (0, 255, 0), 5)

    line_length = []

    for line in linelist:
        line_length.append(math.dist(line[0], line[1]))

    try:
        dashed_line = linelist[line_length.index(max(line_length))]
    except ValueError:
        dashed_line = dashed_line_previous

    m, b = slopeIntercept(dashed_line)
    initial = (int((height * 0.6 - b) / m), int(height * 0.6))
    final = (int((height - b) / m), height)
    detected_line = cv2.line(detected_line, initial, final, (0, 0, 255), 5)

    return detected_line, solid_line, dashed_line

In [ ]:
# Object detection function
def objectDetection(img, net):
    h, w = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
            cv2.rectangle(img, (startX, startY), (endX, endY), (255, 0, 0), 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(img, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    return img

In [ ]:
# Main video processing loop
video = cv2.VideoCapture("straight_lane.mp4")
out = cv2.VideoWriter('straight_lane_detection_output.mp4', cv2.VideoWriter_fourcc(*'XVID'), 25, (960, 540))
solid_line_previous = None
dashed_line_previous = None
print("Generating video output...\n")

while True:
    isTrue, img = video.read()
    if not isTrue:
        break

    processed_img = preprocessing(img)
    height, width = processed_img.shape
    polygon = [(int(width * 0.1), height), (int(width * 0.45), int(height * 0.6)),
               (int(width * 0.55), int(height * 0.6)), (int(0.95 * width), height)]
    masked_img = regionOfInterest(processed_img, polygon)

    detected_lines, solid_line, dashed_line = lineDetection(img, masked_img, solid_line_previous, dashed_line_previous)
    solid_line_previous = solid_line
    dashed_line_previous = dashed_line

    # Object detection on the current frame
    detected_img = objectDetection(detected_lines, net)

    # Write the frame with both lane and object detection
    out.write(detected_img)

out.release()
video.release()
cv2.destroyAllWindows()
print("Video output generated with lane and object detection.\n")

Generating video output...

Video output generated with lane and object detection.



In [ ]:
# Download the yolov3.cfg file
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg

# Download the yolov3.weights file (this file is large, around 240 MB)
!wget https://pjreddie.com/media/files/yolov3.weights


--2024-10-06 19:11:42--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      

2024-10-06 19:11:42 (51.5 MB/s) - ‘yolov3.cfg’ saved [8342/8342]

--2024-10-06 19:11:42--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  24.0MB/s    in 15s     

2024-10-06 19:1

In [ ]:
import cv2

# Load YOLOv3 model with the downloaded config and weights
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Get the names of the output layers
layer_names = net.getLayerNames()

# Fix for the getUnconnectedOutLayers() method
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

print("YOLOv3 model loaded successfully.")


YOLOv3 model loaded successfully.


In [ ]:
import cv2

# Load the pre-trained YOLOv3 model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Get the names of all the layers in the network
layer_names = net.getLayerNames()

# Get the names of the output layers, handling both 1D and 2D outputs correctly
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

print("Output layers:", output_layers)


Output layers: ['yolo_82', 'yolo_94', 'yolo_106']


In [ ]:
import threading

In [ ]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

In [ ]:
# Get the names of the output layers
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]


In [ ]:
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names


--2024-10-06 19:12:35--  https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625 [text/plain]
Saving to: ‘coco.names’

coco.names          100%[===================>]     625  --.-KB/s    in 0s      

2024-10-06 19:12:35 (32.0 MB/s) - ‘coco.names’ saved [625/625]



In [ ]:
# Load the COCO dataset labels
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]


In [ ]:
# Function to detect objects
def detectObjects(frame):
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    return boxes, confidences, class_ids, indexes

In [ ]:
# Function to draw detected objects
def drawDetectedObjects(frame, boxes, confidences, class_ids, indexes):
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)  # Bounding box color
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    return frame

In [ ]:
# Lane detection functions (as previously defined)
def preprocessing(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gblur = cv2.GaussianBlur(gray, (5, 5), 0)
    white_mask = cv2.threshold(gblur, 200, 255, cv2.THRESH_BINARY)[1]
    lower_yellow = np.array([0, 100, 100])
    upper_yellow = np.array([210, 255, 255])
    yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    return mask

In [ ]:
def regionOfInterest(img, polygon):
    mask = np.zeros_like(img)
    x1, y1 = polygon[0]
    x2, y2 = polygon[1]
    x3, y3 = polygon[2]
    x4, y4 = polygon[3]
    m1 = (y2 - y1) / (x2 - x1)
    m2 = (y3 - y2) / (x3 - x2)
    m3 = (y4 - y3) / (x4 - x3)
    m4 = (y4 - y1) / (x4 - x1)
    b1 = y1 - m1 * x1
    b2 = y2 - m2 * x2
    b3 = y3 - m3 * x3
    b4 = y4 - m4 * x4

    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if i >= m1 * j + b1 and i >= m2 * j + b2 and i >= m3 * j + b3 and i <= m4 * j + b4:
                mask[i][j] = 1

    masked_img = np.multiply(mask, img)
    return masked_img

In [ ]:
def warp(img, source_points, destination_points, destn_size):
    matrix = cv2.getPerspectiveTransform(source_points, destination_points)
    warped_img = cv2.warpPerspective(img, matrix, destn_size)
    return warped_img


In [ ]:
def unwarp(img, source_points, destination_points, source_size):
    matrix = cv2.getPerspectiveTransform(destination_points, source_points)
    unwarped_img = cv2.warpPerspective(img, matrix, source_size)
    return unwarped_img

In [ ]:
def fitCurve(img):
    histogram = np.sum(img[img.shape[0] // 2:, :], axis=0)
    midpoint = int(histogram.shape[0] / 2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    nwindows = 50
    margin = 100
    minpix = 50
    window_height = int(img.shape[0] / nwindows)
    y, x = img.nonzero()
    leftx_current = leftx_base
    rightx_current = rightx_base
    left_lane_indices = []
    right_lane_indices = []

    for window in range(nwindows):
        win_y_low = img.shape[0] - (window + 1) * window_height
        win_y_high = img.shape[0] - window * window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        good_left_indices = ((y >= win_y_low) & (y < win_y_high) & (x >= win_xleft_low) & (x < win_xleft_high)).nonzero()[0]
        good_right_indices = ((y >= win_y_low) & (y < win_y_high) & (x >= win_xright_low) & (x < win_xright_high)).nonzero()[0]
        left_lane_indices.append(good_left_indices)
        right_lane_indices.append(good_right_indices)
        if len(good_left_indices) > minpix:
            leftx_current = int(np.mean(x[good_left_indices]))
        if len(good_right_indices) > minpix:
            rightx_current = int(np.mean(x[good_right_indices]))

    left_lane_indices = np.concatenate(left_lane_indices)
    right_lane_indices = np.concatenate(right_lane_indices)
    leftx = x[left_lane_indices]
    lefty = y[left_lane_indices]
    rightx = x[right_lane_indices]
    righty = y[right_lane_indices]
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    return left_fit, right_fit

In [ ]:
def findPoints(img_shape, left_fit, right_fit):
    ploty = np.linspace(0, img_shape[0] - 1, img_shape[0])
    left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])

    return pts_left, pts_right

In [ ]:
def fillCurves(img_shape, pts_left, pts_right):
    pts = np.hstack((pts_left, pts_right))
    img = np.zeros((img_shape[0], img_shape[1], 3), dtype='uint8')
    cv2.fillPoly(img, np.int_([pts]), (0, 0, 255))
    return img

In [ ]:
def oneToThreeChannel(binary):
    img = np.zeros((binary.shape[0], binary.shape[1], 3), dtype='uint8')
    img[:, :, 0] = binary
    img[:, :, 1] = binary
    img[:, :, 2] = binary
    return img

In [ ]:
def drawCurves(img, pts_left, pts_right):
    img = oneToThreeChannel(img)
    cv2.polylines(img, np.int32([pts_left]), isClosed=False, color=(0, 0, 255), thickness=10)
    cv2.polylines(img, np.int32([pts_right]), isClosed=False, color=(0, 255, 255), thickness=10)
    return img

In [ ]:
def concatenate(img1, img2, img3, img4, img5):
    offset = 50
    img3 = setOffset(img3, offset)
    img4 = setOffset(img4, offset)
    img5 = setOffset(img5, offset)
    return np.concatenate((img1, img2, img3, img4, img5), axis=1)

In [ ]:
def setOffset(img, offset):
    img = np.zeros((img.shape[0] + offset, img.shape[1], img.shape[2]), dtype='uint8')
    img[offset:, :, :] = img
    return img

In [ ]:
# Main processing loop
video = cv2.VideoCapture("test_input.mp4")
height, width, _ = frame.shape
fps = video.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter('test_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
# Change the video writer to output in mp4 format
print("Generating video output with lane and object detection...\n")

while True:
    isTrue, frame = video.read()
    if not isTrue:
        break

    # Detect lanes
    processed_img = preprocessing(frame)
    height, width = processed_img.shape
    polygon = [(int(width*0.15), int(height*0.94)),
               (int(width*0.45), int(height*0.62)),
               (int(width*0.58), int(height*0.62)),
               (int(0.95*width), int(0.94*height))]

    masked_img = regionOfInterest(processed_img, polygon)
    source_points = np.float32([[int(width*0.49), int(height*0.62)],
                                 [int(width*0.58), int(height*0.62)],
                                 [int(width*0.15), int(height*0.94)],
                                 [int(0.95*width), int(0.94*height)]])
    destination_points = np.float32([[0,0], [400,0], [0, 960], [400, 960]])
    warped_img_size = (400, 960)

    warped_img = warp(masked_img, source_points, destination_points, warped_img_size)
    kernel = np.ones((11,11), np.uint8)
    opening = cv2.morphologyEx(warped_img, cv2.MORPH_CLOSE, kernel)

    left_fit, right_fit = fitCurve(opening)
    pts_left, pts_right = findPoints(warped_img.shape, left_fit, right_fit)
    fill_curves = fillCurves(warped_img.shape, pts_left, pts_right)
    unwarped_fill_curves = unwarp(fill_curves, source_points, destination_points, (width, height))
    lane_overlay = cv2.addWeighted(frame, 1, unwarped_fill_curves, 1, 0)

    # Detect objects
    boxes, confidences, class_ids, indexes = detectObjects(frame)
    frame_with_objects = drawDetectedObjects(frame, boxes, confidences, class_ids, indexes)

    # Combine lane detection and object detection
    result = cv2.addWeighted(lane_overlay, 0.6, frame_with_objects, 0.4, 0)

    # Write the frame to output video
    out.write(result)

out.release()
video.release()
print("Video output with lane and object detection generated.\n")

Generating video output with lane and object detection...

Video output with lane and object detection generated.



In [ ]:
import os
print(os.getcwd())


/content


In [ ]:
from google.colab import files

# Download the output video
files.download('test_output.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import cv2
import numpy as np

# Preprocessing function (convert to grayscale, apply Gaussian blur, Canny edge detection)
def preprocessing(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)
    return edges

# Region of interest masking function
def regionOfInterest(img, polygon):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, [polygon], 255)
    masked_img = cv2.bitwise_and(img, mask)
    return masked_img

# Warp image to top-down view
def warp(img, src_points, dst_points, size):
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    warped = cv2.warpPerspective(img, matrix, size)
    return warped

# Unwarp image to normal view
def unwarp(img, src_points, dst_points, size):
    matrix = cv2.getPerspectiveTransform(dst_points, src_points)
    unwarped = cv2.warpPerspective(img, matrix, size)
    return unwarped

# Fit curves using polynomial
def fitCurveOptimized(img, left_fit, right_fit):
    y, x = img.nonzero()
    margin = 100
    left_lane_indices = ((x > (left_fit[0] * y ** 2 + left_fit[1] * y + left_fit[2] - margin)) &
                         (x < (left_fit[0] * y ** 2 + left_fit[1] * y + left_fit[2] + margin)))
    right_lane_indices = ((x > (right_fit[0] * y ** 2 + right_fit[1] * y + right_fit[2] - margin)) &
                          (x < (right_fit[0] * y ** 2 + right_fit[1] * y + right_fit[2] + margin)))

    if len(left_lane_indices) == 0 or len(right_lane_indices) == 0:
        return None, None

    leftx = x[left_lane_indices]
    lefty = y[left_lane_indices]
    rightx = x[right_lane_indices]
    righty = y[right_lane_indices]

    if len(leftx) > 0 and len(rightx) > 0:
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        return left_fit, right_fit
    else:
        return None, None

# Fill lane curves with red color
def fillCurves(img_shape, pts_left, pts_right):
    lane_area = np.zeros(img_shape, dtype=np.uint8)
    pts = np.vstack([pts_left, np.flipud(pts_right)])
    cv2.fillPoly(lane_area, np.int_([pts]), (0, 0, 255))  # Red color for lane marking
    return lane_area

# Object detection function using YOLO
def detectObjects(frame):
    height, width = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)

    net.setInput(blob)
    layerOutputs = net.forward(layer_names)

    boxes, confidences, class_ids = [], [], []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                box = detection[0:4] * np.array([width, height, width, height])
                centerX, centerY, w, h = box.astype("int")
                x = int(centerX - w / 2)
                y = int(centerY - h / 2)
                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    return boxes, confidences, class_ids, indexes

# Draw detected objects
def drawDetectedObjects(frame, boxes, confidences, class_ids, indexes):
    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            color = [int(c) for c in colors[class_ids[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(labels[class_ids[i]], confidences[i])
            cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return frame

# Load YOLO model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
layer_names = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
labels = open("coco.names").read().strip().split("\n")
colors = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")

# Video input and output
video = cv2.VideoCapture("curved_lane.mp4")
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter("curved_output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Lane detection initial variables
left_fit, right_fit = None, None
source_points = np.float32([[200, height], [width-200, height], [width//2+50, height//2], [width//2-50, height//2]])
destination_points = np.float32([[300, height], [width-300, height], [width-300, 0], [300, 0]])
warped_img_size = (width, height)
polygon = np.array([[200, height], [width-200, height], [width//2+50, height//2], [width//2-50, height//2]])

# Main processing loop
frame_count = 0
while True:
    ret, frame = video.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % 2 != 0:  # Skip every other frame for efficiency
        continue

    # Resize frame for faster processing
    small_frame = cv2.resize(frame, (width//2, height//2))

    # Lane detection steps
    processed_img = preprocessing(small_frame)
    masked_img = regionOfInterest(processed_img, polygon)
    warped_img = warp(masked_img, source_points, destination_points, warped_img_size)

    # Check if the previous left_fit and right_fit are available
    if left_fit is not None and right_fit is not None:
        left_fit, right_fit = fitCurveOptimized(warped_img, left_fit, right_fit)

    if left_fit is not None and right_fit is not None:
        # Generating lane points
        ploty = np.linspace(0, height - 1, height)
        left_fitx = left_fit[0] * ploty**2 + left_fit[1] * ploty + left_fit[2]
        right_fitx = right_fit[0] * ploty**2 + right_fit[1] * ploty + right_fit[2]

        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.transpose(np.vstack([right_fitx, ploty]))])

        # Filling curves
        fill_curves = fillCurves((height, width, 3), pts_left, pts_right)
        unwarped_fill_curves = unwarp(fill_curves, source_points, destination_points, (width, height))

        # Overlay the lane marking onto the frame
        lane_overlay = cv2.addWeighted(frame, 1, unwarped_fill_curves, 0.3, 0)
    else:
        lane_overlay = frame  # Fallback to the original frame if no lane detected

    # Perform object detection on the current frame
    boxes, confidences, class_ids, indexes = detectObjects(lane_overlay)
    lane_overlay = drawDetectedObjects(lane_overlay, boxes, confidences, class_ids, indexes)

    # Save to output video
    out.write(lane_overlay)

# Release resources
video.release()
out.release()
cv2.destroyAllWindows()


In [ ]:
import os
print(os.getcwd())


/content


In [ ]:
from google.colab import files

# Download the output video
files.download('curved_output.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>